# Analytic Plan Generator

This notebook uses artificial intelligence to generate analytic plans.

## Background

As described in TC 3-12.2.4.1, "The Analytic Scheme of Maneuver is the plan to collect and analyze technical data to meet specific information requirements. It identifies what data to analyze, how to analyze it, and why it is being analyzed." The analytic scheme of maneuver, or ASOM, consists of the following components:

* **Priority information requirement**:
* **Indicator**:
* **Evidence**:
* **Data**:
* **NAI**:
* **Analytic**:



## Environment Setup

This section sets up the environment. It installs packages necessary to generate the analytic plans, imports modules, initializes helper functions, and finally defines global variables. This section also mounts Google Drive to the runtime and moves into the project folder.

### Install Packages

In [ ]:
!pip install -U -q "google" 1> /dev/null

### Import Modules

In [ ]:
from google.colab import userdata
from google.colab import drive
import json
import os

### Initialize Helper Functions

In [ ]:
def build_asom(attack_chain):
    """
    Builds a list of JSON objects (asom) by processing technique files
    based on the provided attack_chain.

    Args:
        attack_chain (dict): A dictionary where keys are MITRE ATT&CK tactics
                             and values are lists of techniques.

    Returns:
        list: A list of JSON objects (asom) that match the criteria.
    """
    asom = []
    current_directory = os.getcwd() # Or specify a directory if not current

    for tactic, techniques in attack_chain.items():
        for technique in techniques:
            # Sanitize technique name to create a valid filename
            # Technique name might be "Txxxx - Technique Name"
            # Filename should be "Txxxx - Technique Name.json"
            file_name = f"{technique}.json"
            file_path = os.path.join(current_directory, file_name)

            if os.path.exists(file_path):
                try:
                    with open(file_path, 'r') as f:
                        technique_data_list = json.load(f)

                    # Ensure technique_data_list is a list
                    if not isinstance(technique_data_list, list):
                        print(f"Warning: Content of {file_name} is not a list. Skipping.")
                        continue

                    for item in technique_data_list:
                        # Ensure item is a dictionary and has at least one key
                        if not isinstance(item, dict) or not item:
                            print(f"Warning: Invalid item format in {file_name}. Skipping item: {item}")
                            continue

                        # The PIR is the first key in the item dictionary
                        pir_key = next(iter(item)) # Gets the first key

                        # Check if the parent tactic is in the PIR key
                        # The tactic in attack_chain is like "TA001 - Initial Access"
                        # We should check if "TA001" is in the pir_key
                        tactic_id = tactic.split(" - ")[0] # Extracts "TA001"
                        if tactic_id in pir_key:
                            asom.append(item)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON from file: {file_name}")
                except Exception as e:
                    print(f"An error occurred while processing {file_name}: {e}")
            else:
                print(f"File not found for technique: {file_name}")
    return asom

### Define Global Variables

In [ ]:
# Toggle logging on (verbose = True)/off (verbose = False)
verbose = True
# verbose = False

In [ ]:
# Rate limits: https://ai.google.dev/gemini-api/docs/rate-limits
# Pricing: https://ai.google.dev/gemini-api/docs/pricing
# Usage: https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com/metrics?project=gen-lang-client-0497172401
# Note that this notebook is designed to be run in Google Colab. The line below reads the Gemini API key for AI Studio,
# which is configured in the Secrets tab on the left side of the Colab window.
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")
log("Gemii API key loaded.")

[2025-05-08T10:30:03.461506] Gemii API key loaded.


### Mount Google Drive

In [ ]:
# Mount Google Drive and move into the Google AI Studio folder
DRIVE_PATH = "/content/drive"
TECHNIQUES_PATH = "/content/drive/MyDrive/Google AI Studio/techniques"

drive.mount(DRIVE_PATH)
log(f"Google Drive mounted to {DRIVE_PATH}")

os.chdir(TECHNIQUES_PATH)
log(f"Changed directory to {TECHNIQUES_PATH}")

Mounted at /content/drive
[2025-05-08T10:30:19.691343] Google Drive mounted to /content/drive
[2025-05-08T10:30:20.180694] Changed directory to /content/drive/MyDrive/Google AI Studio/techniques


## ASOM Generation

This section first retrieves the latest MITRE ATT&CK Matrix for Enterprise from MITRE's GitHub repository. Then, it builds a data structure that stores each technique as they key with it's parent tactic, the technique's description, and detection suggestions as a nested dictionary as the value.

In [ ]:
# Example attack_chain data structure
attack_chain_data = {
    "TA0001 - Initial Access": [
        "T1190 - Exploit Public-Facing Application",
        "T1566 - Phishing",
        "T1078 - Valid Accounts"
    ],
    "TA0004 - Privilege Escalation": [
        "T1078 - Valid Accounts"
    ]
}

log("Building ASOM...")
resulting_asom = build_asom(attack_chain_data)

print("\nResulting ASOM:")
if resulting_asom:
    for entry in resulting_asom:
        print(json.dumps(entry, indent=4))
else:
    log("ASOM is empty.")

[2025-05-08T10:30:33.531001] MitreAttackData object initialized successfully from 'enterprise-attack.json'.
